# Shelter Dogs
For this project, we play the role of a volunteer data analyst for a shelter dog organization where the management have few questions. 
Data in CSV format is from Kaggle.com and is quite messy. The scope of this project is not to clean the data but rather to answer the questions using SQL. 

The database contains one table:

<h4 id="shelter_dogs"><code>shelter_dogs</code></h3>
<table>
<thead>
<tr>
<th>column</th>
<th>data_type</th>
<th>description</th>
</tr>
</thead>
<tbody>
<tr>
<td><code>ID</code></td>
<td><code>bigint</code></td>
<td>Input ID</td>
</tr>
<tr>
<td><code>name</code></td>
<td><code>text</code></td>
<td>Given name</td>
</tr>
<tr>
<td><code>age</code></td>
<td><code>double</code></td>
<td>Estimate age</td>
</tr>
<tr>
<td><code>sex</code></td>
<td><code>text</code></td>
<td>Sex</td>
</tr>
<tr>
<td><code>breed</code></td>
<td><code>text</code></td>
<td>Breed</td>
</tr>
<tr>
<td><code>date found</code></td>
<td><code>text</code></td>
<td>Date dog was found</td>
</tr>
<tr>
<td><code>adoptable from</code></td>
<td><code>text</code></td>
<td>Date is ready for adoption</td>
</tr>
<tr>
<td><code>posted</code></td>
<td><code>text</code></td>
<td>Date adoption ad is posted</td>
</tr>
<tr>
<td><code>color</code></td>
<td><code>text</code></td>
<td>Fur collor</td>
</tr>
<tr>
<td><code>coat</code></td>
<td><code>text</code></td>
<td>Coat length</td>
</tr>
<tr>
<td><code>size</code></td>
<td><code>text</code></td>
<td>Size when found</td>
</tr>
<tr>
<td><code>neutered</code></td>
<td><code>text</code></td>
<td>Neutered when found</td>
</tr>
<tr>
<td><code>housebroken</code></td>
<td><code>text</code></td>
<td>Housebroken</td>
</tr>
<tr>
<td><code>likes_people</code></td>
<td><code>text</code></td>
<td>Likes people</td>
</tr>
<tr>
<td><code>likes_children</code></td>
<td><code>text</code></td>
<td>Likes children</td>
</tr>
<tr>
<td><code>get_along_males</code></td>
<td><code>text</code></td>
<td>Get along male dogs</td>
</tr>
<tr>
<td><code>get_along_females</code></td>
<td><code>text</code></td>
<td>Get along female dogs</td>
</tr>
<tr>
<td><code>get_along_cats</code></td>
<td><code>text</code></td>
<td>Get along cats</td>
</tr>
<tr>
<td><code>keep_in</code></td>
<td><code>text</code></td>
<td>Where recommended to keep</td>
</tr>
</tbody>
</table>

***1. Upload data to SQL server***

To work with SQL requires a server. I choose an online server rather to work locally. Will use python’s module panda to read the CSV file and then push the data to the server. 


In [1]:
from sqlalchemy import create_engine
import pandas as pd


# Create connection to server
host="sql.freedb.tech"
database="freedb_my_data"
user="freedb_albert"
password="w4F%404b7qdwXUS8&"
port = 3306

def get_connection():
    return create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

# Send data from CSV file to server
df = pd.read_csv('/Users/albert/DataAnalytics/1.Projects&Databases/6.(SQL)Shelter-Dogs/ShelterDogs.csv')
df.to_sql('shelter_dogs',get_connection(), index=False)
print ("Data_Exported")


# Server to df (if needed) 

#    sql = 

#    '''
#    SELECT * 
#    FROM shelter_dogs
#    LIMIT 3;
#    '''

#    df = pd.read_sql(sql, get_connection())
#    print (df)

Data_Exported


***2. Testing the connection and see the data***

Created a new connection with SQLTOOLS in VS Code. 

In [ ]:

DESCRIBE shelter_dogs;

SELECT * FROM shelter_dogs LIMIT 10;

Connection successfully. From the first look can see bad data formatting, misspelling, missing data.

***3. Any record for 'Border Terrier'?***

In [ ]:
SELECT name 
FROM shelter_dogs
WHERE name LIKE '_order%' 

No Border terrier in the name list.

***4. Check for missing data.***

Perform a quick check for missing data.


In [ ]:
SELECT  COUNT(*) AS total_rows,
        COUNT(ID),
        COUNT(name),
        COUNT(age),
        COUNT(sex),
        COUNT(breed),
        COUNT(date_found),
        COUNT(neutered),
        COUNT(housebroken),
        COUNT(likes_people)
FROM shelter_dogs;


***5. Is ID the primary key / unique?***

In [ ]:
SELECT ID, COUNT(ID)
FROM shelter_dogs
GROUP BY ID
ORDER BY COUNT(ID) DESC;


ID not unique. Not great.

***6. Are there duplicate rows?***

In [ ]:
SELECT ID, name, age, sex, breed, date_found, adoptable_from, posted, color, coat, size, neutered, housebroken, likes_people, likes_children, get_along_males, get_along_females, get_along_cats, keep_in
FROM shelter_dogs
GROUP BY ID, name, age, sex, breed, date_found, adoptable_from, posted, color, coat, size, neutered, housebroken, likes_people, likes_children, get_along_males, get_along_females, get_along_cats, keep_in
HAVING COUNT(*) > 1;


No duplicate rows.

***7. Are more puppies or older dogs?***

I am thinking to create 3 groups(bins) of age, up to 10 months for puppy, up to 12 years for adult and over 12 years old category.

In [ ]:

SELECT  category, 
        COUNT(category) as 'count',
        CONCAT(ROUND( COUNT(category)/(SELECT COUNT(age) FROM shelter_dogs), 2),' %') AS percent
        FROM(
                SELECT  CASE WHEN age < 0.84 THEN 'puppy'
                        WHEN age > 12.0 THEN 'old'
                        ELSE 'adult'
                        END AS category,
                        age
                FROM shelter_dogs
            ) AS tab
GROUP BY category
ORDER BY count DESC

***8. How many days pass from when a dog is found up to is ready for adoption.***

One way would be to extract day, month and year. Then create new columns to easily extract date difference in days.

In [ ]:


-- extract number of days and count, where number of days more than 0
SELECT  
        DATEDIFF(adoptable_from_2, date_found_2) as days,
        COUNT(*) as count

FROM(
        -- created new columns for date_found and adaptable_from in date format
        WITH tab AS (

                -- extract day, month, year from date_found and adaptable_from columns
                SELECT  SUBSTRING_INDEX(date_found, '/', 1) AS date_found_month,
                        SUBSTRING_INDEX( SUBSTRING_INDEX(date_found, '/', 2),'/',-1 ) AS date_found_day,
                        SUBSTRING_INDEX(date_found, '/', -1) AS date_found_year,
                        SUBSTRING_INDEX(adoptable_from, '/', 1) AS adoptable_from_month,
                        SUBSTRING_INDEX( SUBSTRING_INDEX(adoptable_from, '/', 2),'/',-1 ) AS adoptable_from_day,
                        SUBSTRING_INDEX(adoptable_from, '/', -1) AS adoptable_from_year
                FROM shelter_dogs   

                )           
        
        SELECT  DATE(CONCAT(date_found_year, '-',date_found_month, '-', date_found_day)) as date_found_2,
                DATE(CONCAT(adoptable_from_year, '-',adoptable_from_month, '-', adoptable_from_day)) as adoptable_from_2 FROM tab
) AS sub

WHERE DATEDIFF(adoptable_from_2, date_found_2) >= 0 -- as they are dates with adoption dates before date found
GROUP BY days
ORDER BY count DESC

***9. Create a new table with dog breeds.***

In [ ]:
SELECT  ROW_NUMBER() OVER (ORDER BY breed) as ID,
        breed
FROM shelter_dogs
WHERE breed NOT LIKE 'Adoptable From:'
GROUP BY breed
HAVING breed NOT LIKE '%Mix%'

***10. Top 3 ranked dog names for both male and female***

In [ ]:
SELECT name, sex, _count, _rank 
FROM (
    SELECT  name, 
            sex, 
            COUNT(name) AS _count,
            RANK() OVER(ORDER BY COUNT(name) DESC) AS _rank
    FROM shelter_dogs
    WHERE sex = 'male'
    GROUP BY name ) AS male_table
WHERE _rank < 4

UNION

SELECT name, sex, _count, _rank 
FROM (
    SELECT  name, 
            sex, 
            COUNT(name) AS _count,
            RANK() OVER(ORDER BY COUNT(name) DESC) AS _rank
    FROM shelter_dogs
    WHERE sex = 'female'
    GROUP BY name ) AS female_table
WHERE _rank < 4;